In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch 
import numpy as np 
import torchvision.transforms as transforms 
import torchvision 
import torch.nn.functional as FUNC

In [0]:
class simple_block(torch.nn.Module):
    #its value is 1 as there is no expansion in simple base block
    expansion = 1
    #planes: desired output plane
    #dim_change: if the input shape has to change,Change the value 
    def __init__(self,input_planes,planes,stride=1,dim_change=None):
        super(simple_block,self).__init__()
        #declaring convolutional layers with batch norms
        #In ResNet18,there are 2 pairs of each residual block has 1 convolution layer and 1 batch normalization layer
        #Values of kernal size and stride (according to paper)
        self.conv_1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
        #Batch Normalization
        self.bn_1   = torch.nn.BatchNorm2d(planes)
        self.conv_2 = torch.nn.Conv2d(planes,planes,stride=1,kernel_size=3,padding=1)
        self.bn_2   = torch.nn.BatchNorm2d(planes)
        #If the input size has to be changed then change this value
        self.dim_change = dim_change
    def forward(self,x):
        #Saving the residue
        res = x
        #First output: first convolve then batch normalize
        output = FUNC.relu(self.bn_1(self.conv_1(x)))
        #Next connected layer output
        output = self.bn_2(self.conv_2(output))
        #Change the dimensions when needed
        if self.dim_change is not None:
            res = self.dim_change(res)
        #Adding the residue with this output
        output += res
        #Then pass through ReLU
        output = FUNC.relu(output)

        return output


In [0]:
class ResNet(torch.nn.Module):
    #num_layers: number of times each block is there
    def __init__(self,block,num_layers,classes=10):
        super(ResNet,self).__init__()
        #First convolution has 64 layers
        self.input_planes = 64
        #Converts 3 channels to 64 channels
        self.conv_1 = torch.nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
        self.bn_1   = torch.nn.BatchNorm2d(64)
        #num_layer = [2,2,2,2] for ResNet18
        self.layer_1 = self._layer(block,64,num_layers[0],stride=1)
        self.layer_2 = self._layer(block,128,num_layers[1],stride=2)
        self.layer_3 = self._layer(block,256,num_layers[2],stride=2)
        self.layer_4 = self._layer(block,512,num_layers[3],stride=2)
        #Average pool using 4x4 filter size with stride 1
        self.averagePool = torch.nn.AvgPool2d(kernel_size=4,stride=1)
        #Fully connected layer has 512 times the number of times the block was extended
        self.f_c    =  torch.nn.Linear(512*block.expansion,classes)
    
    #Forwards the output
    def forward(self,x):
        x = FUNC.relu(self.bn_1(self.conv_1(x)))
        #Pass through the layers
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        #Pass through average pool
        x = FUNC.avg_pool2d(x,4)
        x = x.view(x.size(0),-1)
        x = self.f_c(x)

        return x

    
    #planes: Number of layers you want to have in the block
    def _layer(self,block,planes,num_layers,stride=1):
        dim_change = None
        if stride!=1 or planes != self.input_planes*block.expansion:
            dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes*block.expansion,kernel_size=1,stride=stride),
                                             torch.nn.BatchNorm2d(planes*block.expansion))
        #Using sequential the dimensions can be changed
        Layers =[]
        #Append the first block
        Layers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
        #The input size changes due to above layer
        self.input_planes = planes * block.expansion
        for i in range(1,num_layers):
            Layers.append(block(self.input_planes,planes))
            self.input_planes = planes * block.expansion
        #Converting all the Layers to fully connected layer
        return torch.nn.Sequential(*Layers)
    

In [20]:

def test():
        #To convert data from PIL to tensor
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    #Load train and test set:
    train = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
    train_set = torch.utils.data.DataLoader(train,batch_size=128,shuffle=True)

    test = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
    test_set = torch.utils.data.DataLoader(test,batch_size=128,shuffle=False)
    
    #selecting device
    dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(dev)

    #ResNet-18 
    net = ResNet(simple_block,[2,2,2,2],10)

    net.to(dev)
    #Loss function
    Cost_Func = torch.nn.CrossEntropyLoss()
    #SGD function
    optimizer =  torch.optim.SGD(net.parameters(),lr=0.02,momentum=0.9)

    #Training
    for epoch in range(5):
        print("epoch no:"+str(epoch))
        c_loss = 0
        for i,batch in enumerate(train_set,0):
            data,output = batch
            data,output = data.to(dev),output.to(dev)
            predict = net(data)
            loss = Cost_Func(predict,output)
            c_loss = loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print every 100th batch
            if i%100 == 0:
                print('[epoch no:%d  batch no:%d] loss: %.5f'% (epoch+1,i+1,c_loss/1000))
                c_loss = 0
        correct_Hits=0
        total=0
        for batches in test_set:
            data,output = batches
            data,output = data.to(dev),output.to(dev)
            predict = net(data)
            _,predict = torch.max(predict.data,1)  #returns max as well as its index
            total += output.size(0)
            correct_Hits += (predict==output).sum().item()
        print('Accuracy on epoch ',epoch+1,'= ',str((correct_Hits/total)*100))
        print("------------------------------------------------------")

    correct_Hits=0
    total=0
    #Testing
    for batches in test_set:
        data,output = batches
        data,output = data.to(dev),output.to(dev)
        predict = net(data)
        _,predict = torch.max(predict.data,1)  #returns max as well as its index
        total += output.size(0)
        correct_Hits += (predict==output).sum().item()
    print('Accuracy = '+str((correct_Hits/total)*100))


test()

Files already downloaded and verified
Files already downloaded and verified
cuda:0
epoch no:0
[epoch no:1  batch no:1] loss: 0.00233
[epoch no:1  batch no:101] loss: 0.00148
[epoch no:1  batch no:201] loss: 0.00119
[epoch no:1  batch no:301] loss: 0.00110
Accuracy on epoch  1 =  65.29
------------------------------------------------------
epoch no:1
[epoch no:2  batch no:1] loss: 0.00096
[epoch no:2  batch no:101] loss: 0.00090
[epoch no:2  batch no:201] loss: 0.00083
[epoch no:2  batch no:301] loss: 0.00076
Accuracy on epoch  2 =  75.58
------------------------------------------------------
epoch no:2
[epoch no:3  batch no:1] loss: 0.00062
[epoch no:3  batch no:101] loss: 0.00062
[epoch no:3  batch no:201] loss: 0.00056
[epoch no:3  batch no:301] loss: 0.00053
Accuracy on epoch  3 =  78.94
------------------------------------------------------
epoch no:3
[epoch no:4  batch no:1] loss: 0.00037
[epoch no:4  batch no:101] loss: 0.00042
[epoch no:4  batch no:201] loss: 0.00036
[epoch no:4